In [1]:
import os
import sys
from pathlib import Path
import subprocess
import time

import pyreadr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from opt.database import get_ground_truth
from opt.peak_model import _do_solve
#from convert_rds import convert_datasets, forecast_to_pandas

### Check configuration

First, we check that the current system is correctly configured.
  * R installed (https://www.r-project.org/),
  * MiniZinc installed (https://www.minizinc.org/),
  * MIP solver available.

The notebook's configuration is taken from the config dictionary, defined below. Change the values to match your system's installation. The default configuration assumes R and MiniZinc live on system's `PATH`.

    'R'      -> Path to Rscript executable, e.g. 'C:\Program Files\R\R-4.3.0\bin\Rscript.exe'
    'mzn'    -> Path to MiniZinc executable, e.g. 'C:\Program Files\MiniZinc\minizinc.exe'
    'solver' -> Name of a MILP solver available to MiniZinc, e.g. 'highs', 'cbc', 'gurobi' (if installed).

In [2]:
## Operating System configuration.
config = {
    'R': 'Rscript',
    'mzn_bin': 'minizinc',
    'solver': 'highs',
    'threads': 8,
    'path_dzn': Path('./data/opt_data.dzn'),
    'path_json': Path('./data/opt_out.json'),
}

## Signals to be forecast.
signals = [ 'pv_power', 'load_power' ]

## Forecasting main script
fc_script = Path('./src/main.R')

## Testing functions.
def test_subprogram(executable, response):

    ## Ask the program for its version to test for valid installation.
    cmd = [executable, '--version']
    try:
        proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        assert proc.stdout.decode('utf8').startswith(response)
    except:
        print(f"Sub-program {executable} not found. Check if exists and executable.")

def test_minizinc(executable, model, solver, response):
    cmd = [executable, model, '--solver', solver]
    try:
        proc = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        assert proc.stdout.decode('utf8').startswith(response)
    except:
        print(f"MiniZinc {solver} response not as expected; perhaps the solver is not configured?")

## Test that R and MiniZinc are installed and runnable,
test_subprogram(config['R'], 'Rscript')
test_subprogram(config['mzn_bin'], 'MiniZinc to FlatZinc')

## Test that MiniZinc is able to solve a test model with the indicated solver.
test_minizinc(config['mzn_bin'],
              Path('./opt/test_model.mzn'),
              config['solver'],
              os.linesep.join(['x = 4;', 'y = 8;']))

### Train forecast models

This next cell will train the LightGBM forecasting models using the `src/main.R` main forecasting script. The resulting trained models will be placed in the `./models` folder. Please allow several minutes to run; if trained models are already present, there is no need to run this cell again.

In [3]:
## Where the models will be stored.
fc_model_paths = { signal: Path(f'./forecast/results/{signal}') for signal in signals }

## Create forecast models, if missing.
for signal, fc_path in fc_model_paths.items():
    if not fc_path.exists():

        ## Log the run's output to file.
        fc_logfile = Path(f'./log_{signal}_fcast.log')

        ## Run the Rscript forecasting code on signal.
        print(f"Forecasting {signal}")
        
        st = time.time()
        with open(fc_logfile, 'w') as fp:
            
            ## Run main.R
            cmd = [config['R'], '--vanilla', fc_script, signal]
            proc = subprocess.run(cmd, stdout=fp, stderr=subprocess.STDOUT)

        ed = time.time()
        
        if proc.returncode == 0:
            print(f"Forecasted {signal} in {ed-st:.1f} seconds.")
        else:
            print(f"Forecast script error, check {fc_logfile} (script ran for {ed-st:.1f} seconds).", file=sys.stderr)

Forecasting pv_power


Forecast script error, check log_pv_power_fcast.log (script ran for 168.6 seconds).


Forecasting load_power


Forecast script error, check log_load_power_fcast.log (script ran for 219.7 seconds).


### Slice a subset from the data

At this point, we select one day from the datasets, including the forecast made just at the start of the period. Change the period being forecast by modifying the `start_date`, `end_date`, and `at_time`.

In [ ]:
# Units being forecast.
units = [17,  23,  44,  79,  96, 110, 138, 178, 180, 181, 183, 229, 252, 261, 268, 272]

# When the forecast is for.
start_date = "2022-07-23"
end_date = "2022-07-23"
at_time = ["2022-07-22 23:55:00"]

# Load the ground truth data into memory.
df_pv_true = get_ground_truth(units, 'pv_power', start_date, end_date)
df_load_true = get_ground_truth(units, 'load_power', start_date, end_date)

# Localize our data to UTC.
df_load_true.index = df_load_true.index.tz_localize('utc')
df_pv_true.index = df_pv_true.index.tz_localize('utc')

print("Data loaded")


In [ ]:
df_load_true

In [ ]:
# Units being forecast.
units = [17,  23,  44,  79,  96, 110, 138, 178, 180, 181, 183, 229, 252, 261, 268, 272]

# When the forecast is for.
start_date = "2022-07-23"
end_date = "2022-07-23"
at_time = ["2022-07-23 23:55:00"]

def get_forecast(fc_path, at_time):

    ## Get forecast for _all_ time steps.
    all_forecasts = forecast_to_pandas(fc_path)

    ## Select forecasts made 'at' a given time.
    forecast = all_forecasts.loc[at_time]
    return forecast.reset_index().set_index(['forecast_for', 'unit']).unstack(1)['forecast']


# Load the ground truth data into memory.
df_pv_true = get_ground_truth(units, 'pv_power', start_date, end_date)
df_load_true = get_ground_truth(units, 'load_power', start_date, end_date)

# Localize our data to UTC.
df_load_true.index = df_load_true.index.tz_localize('utc')
df_pv_true.index = df_pv_true.index.tz_localize('utc')

print("Data loaded")

# Load the forecast data into memory.
df_load_fcast = get_forecast(fc_paths['load_power'], at_time)
df_pv_fcast = get_forecast(fc_paths['pv_power'], at_time)

# Localize forecast to UTC.
df_load_fcast.index = df_load_fcast.index.tz_localize('utc')
df_pv_fcast.index = df_pv_fcast.index.tz_localize('utc')

print("Forecast loaded")

## Plot the forecast against the ground truth
fig, (ax1, ax2) = \
      plt.subplots(nrows=1, ncols=2,
                   constrained_layout=True,
                   figsize=(11,3), dpi=200)

locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
formatter = mdates.DateFormatter('%T', tz='Australia/Adelaide')

agg_pv = df_pv_true.sum(axis="columns")
agg_load = df_load_true.sum(axis="columns")
agg_pv_fcast = df_pv_fcast.sum(axis="columns")
agg_load_fcast = df_load_fcast.sum(axis="columns")

ax1.step(x=agg_pv.index, y=agg_pv.values, where='post', label='Truth')
ax1.step(x=agg_pv_fcast.index, y=agg_pv_fcast.values, where='post', label='Forecast')
ax1.set_title('PV generation')
ax1.set_xlabel('Time of day')
ax1.set_ylabel('Power (kW)')
ax1.set_ylim(-5, 55)
ax1.xaxis.set_major_locator(locator)
ax1.xaxis.set_major_formatter(formatter)
ax1.legend()

ax2.step(x=agg_load.index, y=agg_load.values, where='post', label='Truth')
ax2.step(x=agg_load_fcast.index, y=agg_load_fcast.values, where='post', label='Truth')
ax2.set_title('Baseload')
ax2.set_xlabel('Time of day')
ax2.set_ylim(-5, 55)
ax2.xaxis.set_major_locator(locator)
ax2.xaxis.set_major_formatter(formatter)

plt.show()

In [ ]:
# Units being forecast.
units = [17,  23,  44,  79,  96, 110, 138, 178, 180, 181, 183, 229, 252, 261, 268, 272]

# When the forecast is for.
start_date = "2022-07-23"
end_date = "2022-07-23"
at_times = [
    "2022-07-23 00:00:00",
    "2022-07-23 00:30:00",
    "2022-07-23 01:00:00",
    "2022-07-23 01:30:00",
]

def get_forecast(fc_path, at_time):

    ## Get forecast for _all_ time steps.
    all_forecasts = forecast_to_pandas(fc_path)

    ## Select forecasts made 'at' a given time.
    forecast = all_forecasts.loc[at_time]
    return forecast.reset_index().set_index(['forecast_for', 'unit']).unstack(1)['forecast']


# Load the ground truth data into memory.
df_pv_true = get_ground_truth(units, 'pv_power', start_date, end_date)
df_load_true = get_ground_truth(units, 'load_power', start_date, end_date)

# Localize our data to UTC.
df_load_true.index = df_load_true.index.tz_localize('utc')
df_pv_true.index = df_pv_true.index.tz_localize('utc')

print("Data loaded")

for at_time in at_times:
    # Load the forecast data into memory.
    df_load_fcast = get_forecast(fc_paths['load_power'], at_time)
    df_pv_fcast = get_forecast(fc_paths['pv_power'], at_time)

    # Localize forecast to UTC.
    df_load_fcast.index = df_load_fcast.index.tz_localize('utc')
    df_pv_fcast.index = df_pv_fcast.index.tz_localize('utc')

    print("Forecast loaded")

    ## Plot the forecast against the ground truth
    fig, (ax1, ax2) = \
          plt.subplots(nrows=1, ncols=2,
                       constrained_layout=True,
                       figsize=(11,3), dpi=200)

    locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
    formatter = mdates.DateFormatter('%T', tz='Australia/Adelaide')

    agg_pv = df_pv_true.sum(axis="columns")
    agg_load = df_load_true.sum(axis="columns")
    agg_pv_fcast = df_pv_fcast.sum(axis="columns")
    agg_load_fcast = df_load_fcast.sum(axis="columns")

    ax1.step(x=agg_pv.index, y=agg_pv.values, where='post', label='Truth')
    ax1.step(x=agg_pv_fcast.index, y=agg_pv_fcast.values, where='post', label='Forecast')
    ax1.set_title('PV generation')
    ax1.set_xlabel('Time of day')
    ax1.set_ylabel('Power (kW)')
    ax1.set_ylim(-5, 55)
    ax1.xaxis.set_major_locator(locator)
    ax1.xaxis.set_major_formatter(formatter)
    ax1.legend()

    ax2.step(x=agg_load.index, y=agg_load.values, where='post', label='Truth')
    ax2.step(x=agg_load_fcast.index, y=agg_load_fcast.values, where='post', label='Truth')
    ax2.set_title('Baseload')
    ax2.set_xlabel('Time of day')
    ax2.set_ylim(-5, 55)
    ax2.xaxis.set_major_locator(locator)
    ax2.xaxis.set_major_formatter(formatter)

    plt.show()

In [ ]:
## Prepare battery data.
df_batt = pyreadr.read_r('./forecast/data/anonymous_public_battery_data.rds')[None]
df_batt = df_batt.rename(columns={'unit': 'unit_uuid'})
df_batt = df_batt.set_index('unit_uuid')
df_batt = df_batt.astype(float)
df_batt = (df_batt * 1000).astype(int)
df_batt = df_batt.rename(columns={'batt_kwh': 'batt_wh'})

# Battery efficiency.
df_batt['batt_eff'] = 0.9

# Initial State-of-Charge.
df_batt['batt_soc'] = (0.5 * df_batt['batt_wh']).astype(int)

# Keep only the batteries at the indicated units.
df_batt = df_batt.loc[units]

# Decision-making alpha (balances peak load versus customer costs)
config['alpha'] = 0.25

# Calculate optimal schedule for ground truth.
df_sol = _do_solve(df_load_true.fillna(0), df_pv_true.fillna(0), df_batt, config)

# Calculate optimal schedule for forecast.
df_sol_fcast = _do_solve(df_load_fcast.fillna(0), #.resample('5T').ffill().fillna(0),
                         df_pv_fcast.fillna(0), #.resample('5T').ffill().fillna(0),
                         df_batt, config)

# Calculate optimal schedule for interpolate.
df_sol_fcast_i = _do_solve(df_load_fcast.resample('5T').interpolate().fillna(0),
                         df_pv_fcast.resample('5T').interpolate().fillna(0),
                         df_batt, config)

In [ ]:
## Calculate the aggregate load.
base = agg_load - agg_pv
ctrl = agg_load - agg_pv + df_sol.sum(axis="columns")
ctrl_fcast = agg_load - agg_pv + df_sol_fcast.resample('1T').ffill().sum(axis="columns")
ctrl_inter = agg_load - agg_pv + df_sol_fcast_i.resample('1T').ffill().sum(axis="columns")


## Plot the forecast against the ground truth
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = \
      plt.subplots(nrows=2, ncols=3,
                   constrained_layout=True, sharex=True, sharey=True,
                   figsize=(12,4.5), dpi=200)

ax1.step(x=base.index, y=base.values, where='post', label='No control')
ax1.step(x=ctrl.index, y=ctrl.values, where='post', label='Hindsight optimal')
ax1.set_ylabel('VPP power (kW)')
ax1.set_ylim(-55, 55)
ax1.xaxis.set_major_locator(locator)
ax1.xaxis.set_major_formatter(formatter)
ax1.legend()

ax2.step(x=base.index, y=base.values, where='post', label='No control')
ax2.step(x=ctrl_fcast.index, y=ctrl_fcast.values, where='post', label='Forecast (step)')
ax2.set_ylim(-55, 55)
ax2.xaxis.set_major_locator(locator)
ax2.xaxis.set_major_formatter(formatter)
ax2.legend()

ax3.step(x=base.index, y=base.values, where='post', label='No control')
ax3.step(x=ctrl_inter.index, y=ctrl_inter.values, where='post', label='Forecast (interpolate)')
ax3.set_ylim(-55, 55)
ax3.xaxis.set_major_locator(locator)
ax3.xaxis.set_major_formatter(formatter)
ax3.legend()


batt_opt = df_sol.sum(axis="columns")
batt_step = df_sol_fcast.resample('1T').ffill().sum(axis="columns")
batt_inter = df_sol_fcast_i.resample('1T').ffill().sum(axis="columns")

ax4.step(batt_opt.index, batt_opt.values, color='k')
ax4.set_xlabel('Time of day')
ax4.set_ylabel('Batt. power (kW)')
ax4.xaxis.set_major_locator(locator)
ax4.xaxis.set_major_formatter(formatter)

ax5.step(batt_step.index, batt_step.values, color='k')
ax5.set_xlabel('Time of day')
ax5.xaxis.set_major_locator(locator)
ax5.xaxis.set_major_formatter(formatter)

ax6.step(batt_inter.index, batt_inter.values, color='k')
ax6.set_xlabel('Time of day')
ax6.xaxis.set_major_locator(locator)
ax6.xaxis.set_major_formatter(formatter)

plt.show()

In [ ]:
from peak_model import get_SA_tariff

# Load the ground truth data into memory.
df_indiv_ctrl = df_load_true - df_pv_true + df_sol
#df_indiv_ctrl = df_load_true - df_pv_true + df_sol_fcast_i.resample('1T').ffill()
df_indiv_ctrl = df_indiv_ctrl.dropna()

tariff_import, tariff_export = get_SA_tariff(df_indiv_ctrl.index)

energy_import = ((df_indiv_ctrl.values > 0) * (1/60.0) * df_indiv_ctrl.values)
energy_export = ((df_indiv_ctrl.values < 0) * (1/60.0) * df_indiv_ctrl.values)

print(energy_import.sum())
print(energy_export.sum())

df_indiv_costs = np.zeros_like(df_indiv_ctrl.values)
df_indiv_costs += ((df_indiv_ctrl.values > 0) * (1/60.0) * df_indiv_ctrl.values) * np.expand_dims(tariff_import.values, -1)
df_indiv_costs += ((df_indiv_ctrl.values < 0) * (1/60.0) * df_indiv_ctrl.values) * np.expand_dims(tariff_export.values, -1)

print(df_indiv_costs.sum())
print(np.sort(df_indiv_costs.sum(0)))

plt.plot(df_indiv_ctrl.sum(1))
plt.show()

In [ ]:
df_sol_fcast.resample('1T').ffill().sum(axis="columns")[:25]

In [ ]:
df_sol_fcast.sum(axis=1)

In [ ]:
base = agg_load - agg_pv
ctrl = agg_load - agg_pv + df_sol.sum(axis="columns")
#ctrl = agg_load - agg_pv + df_sol_v2.resample('1T').ffill().sum(axis="columns")

plt.step(x=base.index, y=base.values, where='post', label='No control')
plt.step(x=ctrl.index, y=ctrl.values, where='post', label='Hindsight optimal')
#plt.ylim(-50, 55)
plt.legend()
plt.show()

In [ ]:
df_load_fcast.resample('5T').interpolate()

In [ ]:
df_load_fcast